# Install Package

In [ ]:
# 아래 환경 설치 후 반드시 런타임 재시작 할것
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
!pip install mecab-python3

# Import Module

In [1]:
# 기본 패키지
import os
import numpy as np
import pandas as pd
from tqdm import tqdm

# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 정규화 패키지
import re

# 모델링 패키지
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# NLP 패키지
from konlpy.tag import Okt, Mecab
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 경고무시
import warnings
warnings.filterwarnings('ignore')

# Load Data

In [2]:
df = pd.read_csv('Petition_data_advised.csv')
df.head()

,content,label
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1


# Data Preprocessing


In [3]:
def korean_preprocessing(text):
    # 1. 특수문자 제거
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9 ]', '', text)
    
    # 2. 불용어 제거
    tokenizer = Mecab()
    stopwords = ['을', '를', '이', '가', '은', '는', '의', '에', '와', '과', '도', '다', '로', '라', '지', '든지', '어디', '혹시', '어느', '여기', '저기']
    tokens = tokenizer.morphs(text)
    tokens = [token for token in tokens if not token in stopwords]
    
    # 3. 어절 분리
    words = []
    for token in tokens:
        words.extend(token.split())
    
    # 4. 표준화
    words = [word.strip() for word in words]
    words = [word for word in words if len(word) > 1]
    
    # 5. 정규화
    okt = Okt()
    normalized_words = []
    for word in words:
        normalized_words.append(okt.normalize(word))
    
    # 6. 최종 결과 반환
    result = ' '.join(normalized_words)
    return result

In [4]:
df['tokenized'] = df['content'].apply(korean_preprocessing)
df.head()

,content,label,tokenized
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1,AI 이미지 생성기 무분별 사용 악용 위한 규제 관한 청원 AI 이미지 생성기 개발...
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1,아동학 대살 가해자 엄벌 신상 공개 관한 청원 얼마 아동학 대로 해당 아이 삼촌 입...
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1,중도금 가산 금리 인하 시스템 개편 관한 청원 정부 에서 지역 중도금 가산 금리 대...
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1,한전 공기업 송전 시장 민영화 반대 관한 청원 자금난 으로 인한 민간 송전 시장 민...
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1,12 당한 학교 폭력 관한 청원 OO 초등 학교 2003 입학 OO 초등 학교 에서...


#### Train, Test 분리

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df['tokenized'], df['label'], stratify = df['label'], random_state = 42, test_size=0.1)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((485,), (54,), (485,), (54,))

#### 토크나이징 및 패딩작업 진행

In [7]:
def preprocess(df):
    
    # 전처리된 문장에서 토큰 생성
    tokenizer = Tokenizer(oov_token='<UNK>')
    tokenizer.fit_on_texts(df)
    tokens = tokenizer.texts_to_sequences(df)

    df_wordcount = pd.DataFrame(list(tokenizer.word_counts.items()), columns=['word', 'count'])
    wc = df_wordcount[df_wordcount['count']<2].shape[0]
    vocab_size = len(tokenizer.word_index) - wc + 2
    print('단어 집합의 크기:', vocab_size)
    
    # 패딩
    max_seq_len = max([len(token_list) for token_list in tokens])
    padded_tokens = pad_sequences(tokens, maxlen=max_seq_len, padding='post')

    return padded_tokens

In [8]:
X_train = preprocess(X_train)
X_test = preprocess(X_test)

단어 집합의 크기: 7378
단어 집합의 크기: 1804


# Modeling

In [29]:
vocab_size, embedding_dim = 7378, 256
model = tf.keras.Sequential([
    Embedding(vocab_size, embedding_dim),
    LSTM(units=128),
    Dense(units=64, activation='relu'),
    Dense(units=32, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 256)         1888768   
                                                                 
 lstm_3 (LSTM)               (None, 128)               197120    
                                                                 
 dense_9 (Dense)             (None, 64)                8256      
                                                                 
 dense_10 (Dense)            (None, 32)                2080      
                                                                 
 dense_11 (Dense)            (None, 1)                 33        
                                                                 
Total params: 2,096,257
Trainable params: 2,096,257
Non-trainable params: 0
_________________________________________________________________


In [17]:
!pip install keras_metrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
earlystopping = EarlyStopping(monitor="val_loss", patience = 5)

checkpoint = ModelCheckpoint('lstm_kor.h5',
                             save_best_only=True,
                             save_weights_only=True,
                             monitor='val_loss',
                             mode='min',
                             verbose=False)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, callbacks=[checkpoint, earlystopping], batch_size=32, validation_data=(X_test, y_test))

Epoch 1/50
16/16 [==============================] - 8s 315ms/step - loss: 0.5644 - accuracy: 0.7835 - val_loss: 0.5021 - val_accuracy: 0.8148
Epoch 2/50
16/16 [==============================] - 3s 194ms/step - loss: 0.4762 - accuracy: 0.8206 - val_loss: 0.4802 - val_accuracy: 0.8148
Epoch 3/50
16/16 [==============================] - 3s 195ms/step - loss: 0.4720 - accuracy: 0.8206 - val_loss: 0.4803 - val_accuracy: 0.8148
Epoch 4/50
16/16 [==============================] - 3s 198ms/step - loss: 0.4733 - accuracy: 0.8206 - val_loss: 0.4799 - val_accuracy: 0.8148
Epoch 5/50
16/16 [==============================] - 4s 262ms/step - loss: 0.4749 - accuracy: 0.8206 - val_loss: 0.4797 - val_accuracy: 0.8148
Epoch 6/50
16/16 [==============================] - 2s 151ms/step - loss: 0.4714 - accuracy: 0.8206 - val_loss: 0.4812 - val_accuracy: 0.8148
Epoch 7/50
16/16 [==============================] - 3s 169ms/step - loss: 0.4720 - accuracy: 0.8206 - val_loss: 0.4795 - val_accuracy: 0.8148
Epoch 

In [32]:
# 저장된 checkpoint 로드 후 정확도 측정
model.load_weights('lstm_kor.h5')
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 20ms/step - loss: 0.4794 - accuracy: 0.8148


[0.47944125533103943, 0.8148148059844971]

In [33]:
model.predict(X_test)[0]

2/2 [==============================] - 0s 13ms/step


array([0.18742944], dtype=float32)